### Introduction
Experiment with different search algorithms and heuristics for a agent that performs progression search to solve planning problems. Use the results to answer questions about designing planning systems

### Analyze the search complexity as a function of domain size, search algorithm, and heuristic.

- The chart or table includes data for all search & heuristic combinations for air cargo problems 1 and 2
- The chart or table includes data at least one uninformed search, two heuristics with greedy best first search, and two heuristics with A* on air cargo problems 3 and 4
- Report includes at least a one paragraph discussion of these results that analyzes the growth trends as the problem size increases

In [ ]:
import run_search as rs
import _utils as utils
from timeit import default_timer as timer


def main(p_choices, s_choices):
    problems = [rs.PROBLEMS[i-1] for i in map(int, p_choices)]
    searches = [rs.SEARCHES[i-1] for i in map(int, s_choices)]

    result = None
    for pname, problem_fn in problems:
        for sname, search_fn, heuristic in searches:
            problem_instance = problem_fn()
            heuristic_fn = None if not heuristic else getattr(problem_instance, heuristic)
            result = run_search(problem_instance, search_fn, heuristic_fn)
    return result

def run_search(problem, search_function, parameter=None):
    ip = utils.PrintableProblem(problem)
    start = timer()
    if parameter is not None:
        node = search_function(ip, parameter)
    else:
        node = search_function(ip)
    end = timer()
    return (ip, len(node.solution()), (end - start))

problem, plan_length, elapsed_time  = main([1], [1])
print("########## Actions   Expansions   Goal Tests   New Nodes")
print(f"Problem: {problem}")
print(f"Plan length: {plan_length}")
print(f"Elapsed time: {elapsed_time}")


In [21]:
import run_search as rs
import _utils as utils
from timeit import default_timer as timer

from dataclasses import make_dataclass
Experiment = make_dataclass("Experiment", [("Problem_Name", str), ("Search_Algo", str), ("Heuristic", str),
                                           ("Actions", int), ("Expansions", int), ("Goal_Tests", int), ("New_Nodes", int),
                                           ("Plan_Length", int), ("Elapsed_Time", float)])

def main_modified(p_choices, s_choices):
    problems = [rs.PROBLEMS[i-1] for i in map(int, p_choices)]
    searches = [rs.SEARCHES[i-1] for i in map(int, s_choices)]

    results = [] # list of experiments
    for pname, problem_fn in problems:
        for sname, search_fn, heuristic in searches:
            print(sname)
            problem_instance = problem_fn()
            heuristic_fn = None if not heuristic else getattr(problem_instance, heuristic)
            result = run_search_modified(problem_instance, search_fn, heuristic_fn)
            results.append(Experiment(pname, sname, heuristic, len(result[0].actions_list), 
                                      result[0].succs, result[0].goal_tests, result[0].states, result[1], result[2]))
    return results

def run_search_modified(problem, search_function, parameter=None):
    ip = utils.PrintableProblem(problem)
    start = timer()
    if parameter is not None:
        node = search_function(ip, parameter)
    else:
        node = search_function(ip)
    end = timer()
    return (ip, len(node.solution()), (end - start))


In [10]:
search_results  = main_modified([1], [1, 9, 11])
for result in search_results:
    print(result)
    print("---------------------------------------------------------")


Experiment(Problem_Name='Air Cargo Problem 1', Search_Algo='breadth_first_search', Heuristic='', Actions=20, Expansions=43, Goal_Tests=56, New_Nodes=178, Plan_Length=6, Elapsed_Time=0.007488099996407982)
---------------------------------------------------------
Experiment(Problem_Name='Air Cargo Problem 1', Search_Algo='astar_search', Heuristic='h_pg_levelsum', Actions=20, Expansions=28, Goal_Tests=30, New_Nodes=122, Plan_Length=6, Elapsed_Time=1.1714700999946217)
---------------------------------------------------------
Experiment(Problem_Name='Air Cargo Problem 1', Search_Algo='astar_search', Heuristic='h_pg_setlevel', Actions=20, Expansions=33, Goal_Tests=35, New_Nodes=138, Plan_Length=6, Elapsed_Time=5.236961800001154)
---------------------------------------------------------


In [11]:
import pandas as pd

df = pd.DataFrame(search_results)
df.head()

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 1,breadth_first_search,,20,43,56,178,6,0.007488
1,Air Cargo Problem 1,astar_search,h_pg_levelsum,20,28,30,122,6,1.171470
2,Air Cargo Problem 1,astar_search,h_pg_setlevel,20,33,35,138,6,5.236962


In [12]:
all_searches = [x for x in range(1, 12)]
print(my_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [16]:
search_results_1  = main_modified([1], all_searches)

In [17]:
df = pd.DataFrame(search_results_1)
df

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 1,breadth_first_search,,20,43,56,178,6,0.007711
1,Air Cargo Problem 1,depth_first_graph_search,,20,21,22,84,20,0.004123
2,Air Cargo Problem 1,uniform_cost_search,,20,60,62,240,6,0.011405
3,Air Cargo Problem 1,greedy_best_first_graph_search,h_unmet_goals,20,7,9,29,6,0.001880
4,Air Cargo Problem 1,greedy_best_first_graph_search,h_pg_levelsum,20,6,8,28,6,0.475017
5,Air Cargo Problem 1,greedy_best_first_graph_search,h_pg_maxlevel,20,6,8,24,6,0.346950
6,Air Cargo Problem 1,greedy_best_first_graph_search,h_pg_setlevel,20,6,8,28,6,1.990053
7,Air Cargo Problem 1,astar_search,h_unmet_goals,20,50,52,206,6,0.010775
8,Air Cargo Problem 1,astar_search,h_pg_levelsum,20,28,30,122,6,1.153056
9,Air Cargo Problem 1,astar_search,h_pg_maxlevel,20,43,45,180,6,1.276995


In [18]:
search_results_2  = main_modified([2], all_searches)

In [19]:
df2 = pd.DataFrame(search_results_2)
df2

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 2,breadth_first_search,,72,3343,4609,30503,9,2.313699
1,Air Cargo Problem 2,depth_first_graph_search,,72,624,625,5602,619,2.903870
2,Air Cargo Problem 2,uniform_cost_search,,72,5154,5156,46618,9,3.825029
3,Air Cargo Problem 2,greedy_best_first_graph_search,h_unmet_goals,72,17,19,170,9,0.028149
4,Air Cargo Problem 2,greedy_best_first_graph_search,h_pg_levelsum,72,9,11,86,9,10.795064
5,Air Cargo Problem 2,greedy_best_first_graph_search,h_pg_maxlevel,72,27,29,249,9,21.246409
6,Air Cargo Problem 2,greedy_best_first_graph_search,h_pg_setlevel,72,9,11,84,9,46.505007
7,Air Cargo Problem 2,astar_search,h_unmet_goals,72,2467,2469,22522,9,2.508096
8,Air Cargo Problem 2,astar_search,h_pg_levelsum,72,357,359,3426,9,266.773466
9,Air Cargo Problem 2,astar_search,h_pg_maxlevel,72,2887,2889,26594,9,1570.773955


In [20]:
df.to_pickle("./search_results_1.pkl")
df2.to_pickle("./search_results_2.pkl")

In [25]:
search_results_3  = main_modified([3], [1, 2, 3, 4, 5, 6, 7, 8, 9])

breadth_first_search
depth_first_graph_search
uniform_cost_search
greedy_best_first_graph_search
greedy_best_first_graph_search
greedy_best_first_graph_search
greedy_best_first_graph_search
astar_search
astar_search


In [26]:
df3 = pd.DataFrame(search_results_3)
df3

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 3,breadth_first_search,,88,14663,18098,129625,12,11.392577
1,Air Cargo Problem 3,depth_first_graph_search,,88,408,409,3364,392,1.170486
2,Air Cargo Problem 3,uniform_cost_search,,88,18510,18512,161936,12,15.210399
3,Air Cargo Problem 3,greedy_best_first_graph_search,h_unmet_goals,88,25,27,230,15,0.047403
4,Air Cargo Problem 3,greedy_best_first_graph_search,h_pg_levelsum,88,14,16,126,14,22.725623
5,Air Cargo Problem 3,greedy_best_first_graph_search,h_pg_maxlevel,88,21,23,195,13,27.618326
6,Air Cargo Problem 3,greedy_best_first_graph_search,h_pg_setlevel,88,35,37,345,17,220.472328
7,Air Cargo Problem 3,astar_search,h_unmet_goals,88,7388,7390,65711,12,8.966065
8,Air Cargo Problem 3,astar_search,h_pg_levelsum,88,369,371,3403,12,422.298002


In [27]:
df3.to_pickle("./search_results_3.pkl")

In [28]:
search_results_3_9  = main_modified([3], [10])

astar_search


In [29]:
df3_9 = pd.DataFrame(search_results_3_9)
df3_9

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 3,astar_search,h_pg_maxlevel,88,9580,9582,86312,12,7346.606979


In [30]:
df3_9.to_pickle("./search_results_3_9.pkl")

In [32]:
search_results_4  = main_modified([4], [1, 2, 3, 4])

breadth_first_search
depth_first_graph_search
uniform_cost_search
greedy_best_first_graph_search


In [33]:
df4 = pd.DataFrame(search_results_4)
df4

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 4,breadth_first_search,,104,99736,114953,944130,14,101.328007
1,Air Cargo Problem 4,depth_first_graph_search,,104,25174,25175,228849,24132,3372.150481
2,Air Cargo Problem 4,uniform_cost_search,,104,113339,113341,1066413,14,120.116413
3,Air Cargo Problem 4,greedy_best_first_graph_search,h_unmet_goals,104,29,31,280,18,0.063102


In [34]:
df4.to_pickle("./search_results_4.pkl")

In [35]:
search_results_4_4  = main_modified([4], [5])

greedy_best_first_graph_search


In [36]:
df4_4 = pd.DataFrame(search_results_4_4)
df4_4

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 4,greedy_best_first_graph_search,h_pg_levelsum,104,17,19,165,17,41.053186


In [37]:
df4_4.to_pickle("./search_results_4_4.pkl")

In [38]:
search_results_4_5  = main_modified([4], [6])

greedy_best_first_graph_search


In [39]:
df4_5 = pd.DataFrame(search_results_4_5)
df4_5

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 4,greedy_best_first_graph_search,h_pg_maxlevel,104,56,58,580,17,99.347298


In [40]:
df4_5.to_pickle("./search_results_4_5.pkl")

In [42]:
search_results_4_6  = main_modified([4], [7])

greedy_best_first_graph_search


In [43]:
df4_6 = pd.DataFrame(search_results_4_6)
df4_6

,Problem_Name,Search_Algo,Heuristic,Actions,Expansions,Goal_Tests,New_Nodes,Plan_Length,Elapsed_Time
0,Air Cargo Problem 4,greedy_best_first_graph_search,h_pg_setlevel,104,107,109,1164,23,995.940341


In [44]:
df4_6.to_pickle("./search_results_4_6.pkl")